# Семинар 3: Введение в NumPy

<a href="https://colab.research.google.com/github/SergeyMalashenko/MachineLearning_Summer_2023/blob/main/seminars/3/seminar_3_Numpy.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [201]:
import numpy
numpy.__version__

'1.22.1'

In [202]:
import numpy as np

## Встроенная документация

IPython дает вам возможность быстро изучить содержимое модуля, а также документацию к различным функциям (с помощью символа `?`).

Например, чтобы отобразить все содержимое пространства имен NumPy, вы можете набрать следующее:

```ipython
В [3]: np.<TAB>
```

А чтобы отобразить встроенную документацию NumPy, вы можете использовать следующее:

```ipython
In [4]: np?
```

Более подробную документацию, а также учебники и другие ресурсы можно найти на сайте http://www.numpy.org.

# Понимание типов данных в языке Python

Эффективные вычисления, основанные на данных, требуют понимания того, как хранятся данные и как ими манипулировать.
дадим описание и сравним то, как массивы данных обрабатываются в самом языке Python, и как NumPy улучшает это.
д
Пользователей Python часто привлекает простота использования, одной из составляющих которой является динамическая типизация.
В то время как статически типизированные языки, такие как C или Java, требуют явного объявления каждой переменной, динамически типизированные языки, такие как Python, пропускают эту спецификацию. Например, в языке C вы можете указать конкретную операцию следующим образом:

```C
/* Код на C */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

В то время как в Python эквивалентная операция может быть записана следующим образом:

```python
# Python код
результат = 0
for i in range(100):
    result += i
```

Обратите внимание на главное отличие: в C типы данных каждой переменной объявляются явно, а в Python типы определяются динамически. Это означает, что, например, мы можем присвоить любой тип данных любой переменной:

```python
# Python код
x = 4
x = "четыре"
```

Здесь мы поменяли содержимое ``x`` с целого числа на строку. То же самое в C привело бы (в зависимости от настроек компилятора) к ошибке компиляции или другим непредвиденным последствиям:

```C
/* Код на C */
int x = 4;
x = "четыре"; // ОШИБКА
```

Подобная гибкость - одна из составляющих, которая делает Python и другие динамически типизированные языки удобными и простыми в использовании.
Понимание того, *как* это работает, является важной частью обучения эффективному анализу данных с помощью Python.
Но гибкость типов также указывает на то, что переменные Python - это не просто их значение; они также содержат дополнительную информацию о типе значения.

## Целое число в Python - это больше, чем просто целое число

Стандартная реализация Python написана на языке C.
Это означает, что каждый объект Python - это просто умело замаскированная структура языка C, которая содержит не только свое значение, но и другую информацию. Например, когда мы определяем целое число в Python, например, ``x = 10000``, ``x`` - это не просто "сырое" целое число. На самом деле это указатель на составную структуру C, которая содержит несколько значений.
Просматривая исходный код Python 3.4, мы обнаруживаем, что определение типа integer (long) фактически выглядит следующим образом (после расширения макросов C):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Одно целое число в Python 3.4 на самом деле содержит четыре части:

- ``ob_refcnt``, счетчик ссылок, который помогает Python тихо обрабатывать выделение и удаление памяти
- ``ob_type``, который кодирует тип переменной
- ``ob_size``, который определяет размер следующих членов данных
- ``ob_digit``, который содержит фактическое целочисленное значение, которое, как мы ожидаем, будет представлять переменная Python.

Это означает, что хранение целого числа в Python требует определенных затрат по сравнению с целым числом в компилируемом языке, таком как C

![Integer Memory Layout](images/cint_vs_pyint.png)

Здесь ``PyObject_HEAD`` - это часть структуры, содержащая счетчик ссылок, код типа и другие части, упомянутые ранее.

Обратите внимание на разницу: целое число в языке Си - это, по сути, метка для позиции в памяти, байты которой кодируют целочисленное значение.
Целое число в Python - это указатель на позицию в памяти, содержащий всю информацию об объекте Python, включая байты, которые содержат целочисленное значение.
Эта дополнительная информация в целочисленной структуре Python - то, что позволяет кодировать Python так свободно и динамично.
Однако за всю эту дополнительную информацию в типах Python приходится платить, что становится особенно очевидным в структурах, объединяющих множество таких объектов.

## Список Python - это больше, чем просто список

Давайте теперь рассмотрим, что происходит, когда мы используем структуру данных Python, в которой хранится много объектов Python.
Стандартным многоэлементным контейнером с возможностью изменения в Python является список.
Мы можем создать список целых чисел следующим образом:

In [203]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [204]:
type(L[0])

int

In [205]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [206]:
type(L2[0])

str

In [207]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

Но за эту гибкость приходится платить: чтобы обеспечить гибкость типов, каждый элемент списка должен содержать информацию о своем типе, количество ссылок и другую информацию - то есть, каждый элемент является полным объектом Python.
В особом случае, когда все переменные имеют один тип, большая часть этой информации является избыточной: гораздо эффективнее хранить данные в массиве фиксированного типа.

![Array Memory Layout](images/array_vs_list.png)

На уровне реализации массив, по сути, содержит один указатель на один непрерывный блок данных.
Список Python, с другой стороны, содержит указатель на блок указателей, каждый из которых, в свою очередь, указывает на полный объект Python, такой как целое число Python, которое мы рассматривали ранее.
Опять же, преимущество списка заключается в гибкости: поскольку каждый элемент списка представляет собой полную структуру, содержащую как данные, так и информацию о типе, список может быть заполнен данными любого желаемого типа.
Массивы фиксированного типа в стиле NumPy лишены такой гибкости, но гораздо более эффективны для хранения и манипулирования данными.

In [208]:
import numpy as np

## Создание массива из списка Python

In [209]:
# Целочисленный массив
np.array([1, 4, 2, 5, 3])
# Массив чисел с плавающей точкой
np.array([3.14, 4, 2, 3])
np.array([1, 2, 3, 4], dtype='float32')
# Многомерный массив из вложенных списков
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

## Создание массивов с нуля

In [210]:
# Создание массива данных, заполненого нулями
np.zeros(10, dtype=int)
# Создание массива 3x5, заполенного единицами
np.ones((3, 5), dtype=float)
# Создание массива 3x5, заполенного 3.14
np.full((3, 5), 3.14)
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))
# Create a 3x3 identity matrix
np.eye(3)
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([0.32862676, 0.67780119, 0.84270079])

### NumPy стандартные типы данных

Стандартные типы данных NumPy перечислены в следующей таблице.
Обратите внимание, что при построении массива их можно указать с помощью строки:

```python
np.zeros(10, dtype='int16')
```

Или с помощью связанного объекта NumPy:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	 | Description |
|-------------|-------------|
| `bool_`     | Boolean (True or False) stored as a byte |
| `int_`      | Default integer type (same as C `long`; normally either `int64` or `int32`)| 
| `intc`      | Identical to C `int` (normally `int32` or `int64`)| 
| `intp`      | Integer used for indexing (same as C `ssize_t`; normally either `int32` or `int64`)| 
| `int8`      | Byte (–128 to 127)| 
| `int16`     | Integer (–32768 to 32767)|
| `int32`     | Integer (–2147483648 to 2147483647)|
| `int64`     | Integer (–9223372036854775808 to 9223372036854775807)| 
| `uint8`     | Unsigned integer (0 to 255)| 
| `uint16`    | Unsigned integer (0 to 65535)| 
| `uint32`    | Unsigned integer (0 to 4294967295)| 
| `uint64`    | Unsigned integer (0 to 18446744073709551615)| 
| `float_`    | Shorthand for `float64`| 
| `float16`   | Half-precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| `float32`   | Single-precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| `float64`   | Double-precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| `complex_`  | Shorthand for `complex128`| 
| `complex64` | Complex number, represented by two 32-bit floats| 
| `complex128`| Complex number, represented by two 64-bit floats| 

# Основы работы с массивами NumPy

Работа с данными в Python практически синонимична работе с массивами NumPy: даже такие новые инструменты, как Pandas, построены вокруг массива NumPy.

Здесь мы рассмотрим несколько категорий основных манипуляций с массивами:

- *Атрибуты массивов*: Определение размера, формы, объема памяти и типов данных массивов.
- *Индексация массивов*: Получение и установка значения отдельных элементов массива
- *Разбиение массивов*: Получение и установка меньших подмассивов в большом массиве.
- *Переформирование массивов*: Изменение формы заданного массива
- *Соединение и разделение массивов*: объединение нескольких массивов в один и разбиение одного массива на множество.

## Аттрибуты массивов NumPy

In [211]:
x1 = np.random.randint(10, size=6        )  # Одномерный массив
x2 = np.random.randint(10, size=(3, 4)   )  # Двухмерных массив
x3 = np.random.randint(10, size=(3, 4, 5))  # Трехмерный массив

Каждый массив имеет атрибуты ``ndim`` (количество измерений), ``shape`` (размер каждого измерения) и ``size`` (общий размер массива):

In [212]:
print("x3 ndim: ", x3.ndim )
print("x3 shape:", x3.shape)
print("x3 size: ", x3.size )

x3 ndim:  3
x3 shape: (3, 4, 5)
x3 size:  60


Другим полезным атрибутом является ``dtype``, тип данных массива

In [213]:
print("dtype:", x3.dtype)

dtype: int64


Другие атрибуты включают ``itemsize``, где указан размер (в байтах) каждого элемента массива, и ``nbytes``, где указан общий размер (в байтах) массива.

In [214]:
print("itemsize:", x3.itemsize, "bytes")
print("nbytes:"  , x3.nbytes  , "bytes")

itemsize: 8 bytes
nbytes: 480 bytes


## Индексирование массивов: доступ к отдельным элементам

In [215]:
x1

array([8, 3, 7, 5, 1, 7])

In [216]:
x1[0]

8

In [217]:
x1[-2]

1

In [218]:
x2

array([[1, 3, 9, 6],
       [2, 2, 6, 8],
       [5, 0, 4, 5]])

In [219]:
x2[0, 0]

1

In [220]:
x2[0, -2] = 12
x2

array([[ 1,  3, 12,  6],
       [ 2,  2,  6,  8],
       [ 5,  0,  4,  5]])

In [221]:
x1[-2] = 3.14159  # будет усечено!
x1

array([8, 3, 7, 5, 3, 7])

## Сечение массива: доступ к подмассивам

Можно квадратные скобки использовать для доступа к подмассивам с помощью нотации *slice*, обозначенной символом двоеточия (``:``).
Синтаксис сечения NumPy повторяет стандартный список Python; чтобы получить доступ к подмассива массива ``x`` используется следующий синтаксис:
```python
x[start:stop:step]
```
Если какой-либо из этих параметров не указан, то по умолчанию используются значения ``start=0``, ``stop=``*``размер размерности``*, ``step=1``.
Мы рассмотрим доступ к подмассивам в одном измерении и в нескольких измерениях.

### Одномерный подмассив

In [222]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [223]:
x[:5]  
x[5:]
x[4:7]
x[::2]
x[5::-2]

array([5, 3, 1])

### Многомерный подмассив

Многомерные сечения работают аналогичным образом, при этом несколько сечений разделяются запятыми.

In [224]:
x2

array([[ 1,  3, 12,  6],
       [ 2,  2,  6,  8],
       [ 5,  0,  4,  5]])

In [225]:
x2[:2, :3]

array([[ 1,  3, 12],
       [ 2,  2,  6]])

In [226]:
x2[::-1, ::-1]

array([[ 5,  4,  0,  5],
       [ 8,  6,  2,  2],
       [ 6, 12,  3,  1]])

#### Доступ к строкам и столбцам массива

Одной из часто встречающихся процедур является доступ к отдельным строкам или столбцам массива.

In [227]:
print(x2[:, 0])

[1 2 5]


In [228]:
print(x2[0, :])

[ 1  3 12  6]


In [229]:
print(x2[0]) 

[ 1  3 12  6]


### Подмассивы как представления без копирования

Одна важная и чрезвычайно полезная вещь, которую нужно знать о сечении массивов, заключается в том, что они возвращают *представления (view)*, а не *копии (copy)* данных массива.
Это одна из областей, в которой нарезка массивов NumPy отличается от нарезки списков Python: в списках нарезки будут копиями.

In [230]:
print(x2)

[[ 1  3 12  6]
 [ 2  2  6  8]
 [ 5  0  4  5]]


In [231]:
x2_sub = x2[:2, :2]
print(x2_sub)

[[1 3]
 [2 2]]


In [232]:
x2_sub[0, 0] = 99
print(x2_sub)

[[99  3]
 [ 2  2]]


In [233]:
print(x2)

[[99  3 12  6]
 [ 2  2  6  8]
 [ 5  0  4  5]]


Это поведение по умолчанию на самом деле довольно полезно: при работе с большими наборами данных мы можем получать доступ и обрабатывать их части без необходимости копировать основной буфер данных.

### Создание копий массивов

Несмотря на приятные возможности представления массивов, иногда полезно явно копировать данные внутри массива или подмассива. Это проще всего сделать с помощью метода ``copy()``.

In [234]:
x2_sub_copy = x2[:2, :2].copy()
print(x2_sub_copy)

[[99  3]
 [ 2  2]]


In [235]:
x2_sub_copy[0, 0] = 42
print(x2_sub_copy)

[[42  3]
 [ 2  2]]


In [236]:
print(x2)

[[99  3 12  6]
 [ 2  2  6  8]
 [ 5  0  4  5]]


## Изменение формы (reshaping) массивов

Еще один полезный тип операций - изменение формы массивов.
Наиболее гибко это можно сделать с помощью метода ``reshape``.
Например, если вы хотите поместить числа от 1 до 9 в сетку $3 \times 3$, вы можете сделать следующее:

In [237]:
grid = np.arange(1, 10).reshape((3, 3))
print(grid)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


Размер исходного массива должен соответствовать размеру массива c измененной формой. 
По возможности, метод ``reshape`` будет использовать представление исходного массива без копирования, но в случае с несмежными буферами памяти это не всегда так.

Другим распространенным шаблоном изменение формы является преобразование одномерного массива в двумерную матрицу строк или столбцов.
Это можно сделать с помощью метода ``reshape`` или проще - используя ключевое слово ``newaxis`` в операции сечения.

In [238]:
x = np.array([1, 2, 3])

# row vector via reshape
x.reshape((1, 3))

array([[1, 2, 3]])

In [239]:
x[np.newaxis, :]

array([[1, 2, 3]])

In [240]:
x.reshape((3, 1))

array([[1],
       [2],
       [3]])

In [241]:
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

## Конкатенация и разбиение массивов

Все предыдущие процедуры работали с одиночными массивами. 
Можно также объединить несколько массивов в один и, наоборот, разделить один массив на несколько массивов.

### Конкатенация массивов

Конкатенация, или объединение двух массивов в NumPy, в основном выполняется с помощью процедур ``np.concatenate``, ``np.vstack`` и ``np.hstack``.

In [242]:
x = np.array([ 1,  2,  3])
y = np.array([ 3,  2,  1])
z =          [99, 99, 99]
print(np.concatenate([x, y, z]))

[ 1  2  3  3  2  1 99 99 99]


In [243]:
grid = np.array([[1, 2, 3],[4, 5, 6]])

In [244]:
np.concatenate([grid, grid], axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

In [245]:
np.concatenate([grid, grid], axis=1)

array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]])

Для работы с массивами смешанных размеров может быть более понятным использование функций ``np.vstack`` вдоль первой оси и ``np.hstack`` вдоль второй оси, ``np.dstack`` вдоль третьей оси.

In [246]:
x_1 = np.array([1, 2, 3])
x_2 = np.array([[9, 8, 7],[6, 5, 4]])

# vertically stack the arrays
np.vstack([x_1, x_2])

array([[1, 2, 3],
       [9, 8, 7],
       [6, 5, 4]])

In [247]:
x_3 = np.array([[99],[99]])
np.hstack([x_2, x_3])

array([[ 9,  8,  7, 99],
       [ 6,  5,  4, 99]])

### Разбиение массивов

Противоположностью конкатенации является разбиение, которое реализуется функциями ``np.split``, ``np.hsplit``, ``np.vsplit`` и ``np.dsplit``.
Для каждой из них мы можем передать список индексов, задающих точки разбиения.

In [248]:
x0 = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3 = np.split(x0, [3, 5])
print(x1, x2, x3)

[1 2 3] [99 99] [3 2 1]


In [249]:
grid = np.arange(16).reshape((4, 4))
grid

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [250]:
upper, lower = np.vsplit(grid, [2])
print(upper)
print(lower)

[[0 1 2 3]
 [4 5 6 7]]
[[ 8  9 10 11]
 [12 13 14 15]]


In [251]:
left, right = np.hsplit(grid, [2])
print(left)
print(right)

[[ 0  1]
 [ 4  5]
 [ 8  9]
 [12 13]]
[[ 2  3]
 [ 6  7]
 [10 11]
 [14 15]]


# Вычисления над массивами NumPy: Универсальные функции (UFuncs)

Вычисления с массивами NumPy могут быть очень быстрыми, а могут быть очень медленными.
Ключ к тому, чтобы сделать их быстрыми, - использование *векторных* операций, обычно реализуемых через *универсальные функции* (ufuncs) NumPy.
В этом разделе обосновывается необходимость использования ufuncs в NumPy, с помощью которых можно сделать повторяющиеся вычисления над элементами массива гораздо более эффективными.
Затем будут представлены многие из наиболее распространенных и полезных арифметических функций, доступных в пакете NumPy.

In [252]:
x = np.arange(9).reshape((3, 3))

## UFuncs в NumPy

UFuncs существуют в двух вариантах: *унарные (unary) ufuncs*, которые принимают один массив, и *бинарные (binary) ufuncs*, которые принимают пару массивов.

### Арифметика массивов

NumPy's ufuncs используют арифметические операторы Python: сложение, вычитание, умножение и деление.

In [253]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


Существует также унарные ufunc для отрицания, оператор ``**`` для возведения в степень и оператор ``%`` для вычисления остатка от деления.

In [254]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


Кроме того, ufuncs можно комбинировать, при этом соблюдается стандартный порядок действий.

In [255]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

В следующей таблице перечислены арифметические операторы, реализованные в NumPy.

| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|

### Абсолютное значение

In [256]:
x = np.array([-2, -1, 0, 1, 2])

print( np.absolute(x) )
print( np.abs     (x) )

[2 1 0 1 2]
[2 1 0 1 2]


### Тригонометрические функции

In [257]:
theta = np.linspace(0, np.pi, 3)

print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


In [258]:
x = [-1, 0, 1]

print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Экспоненты и логарифмы

In [259]:
x = [1, 2, 3]

print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


In [260]:
x = [0, 0.001, 0.01, 0.1]

print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


### Специализированные ufuncs (SciPy)

В NumPy доступно множество других ufuncs, включая гиперболические функции, побитовую арифметику, операторы сравнения, преобразование из радиан в градусы, округление и остатки, и многое другое.

Другим отличным источником более специализированных ufincs является подмодуль ``scipy.special``.

In [261]:
from scipy import special

In [262]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]

print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [263]:
x = np.array([0, 0.3, 0.7, 1.0])

print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


## Расширенные возможности Ufunc

### Определение выходного массива

Для больших вычислений иногда полезно иметь возможность указать массив, в котором будет храниться результат вычисления.
Вместо того чтобы создавать временный массив, можно записать результаты вычислений непосредственно в то место памяти, где вы хотели бы их видеть.
Для всех ufuncs это можно сделать с помощью аргумента ``out`` функции.

In [264]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[ 0. 10. 20. 30. 40.]


In [265]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


### Агрегаты

Для бинарных ufunc есть несколько интересных агрегатов, которые можно вычислить непосредственно из объекта.
Например, если мы хотим *уменьшить* массив с помощью определенной операции, мы можем использовать метод ``reduce`` любой ufunc.
Метод reduce многократно применяет заданную операцию к элементам массива, пока не останется только один результат.

Вызов ``reduce`` на ufunc ``add`` возвращает сумму всех элементов массива.

In [266]:
x = np.arange(1, 6)

In [267]:
np.add.reduce(x)

15

In [268]:
np.multiply.reduce(x)

120

Если мы хотим хранить все промежуточные результаты вычислений, мы можем вместо этого использовать ``accumulate``.

In [269]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [270]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

Обратите внимание, что для этих конкретных случаев существуют специальные функции NumPy для вычисления результатов (``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``).

### Тензорное произведение

Любая ufunc может вычислить результат прямого произведения двух различных входов, используя метод ``outer``.

In [271]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

# Агрегации: минимум, максимум и другие функции агрегирования

Часто, когда приходится сталкиваться с большим количеством данных, первым шагом является вычисление сводной статистики для рассматриваемых данных.
Возможно, самыми распространенными сводными статистиками являются среднее значение и стандартное отклонение, которые позволяют суммировать "типичные" значения в наборе данных, но полезны и другие агрегаты (сумма, произведение, медиана, минимум и максимум, квантили и т.д.).

NumPy имеет быстрые встроенные функции агрегирования для работы с массивами.

### Минимум, максимум, сумма

In [272]:
x_1 = np.random.rand(1000000)

In [273]:
np.sum(x_1), np.min(x_1), np.max(x_1)

(499264.5628535573, 3.621641532225439e-07, 0.9999960588030166)

In [274]:
x_1.min(), x_1.max(), x_1.sum()

(3.621641532225439e-07, 0.9999960588030166, 499264.5628535573)

In [275]:
x_3 = np.random.random((3, 5, 7))

In [276]:
x_3.sum()

49.89733103100551

In [277]:
x_3.max(axis=1)

array([[0.93321107, 0.82128085, 0.93456308, 0.78096245, 0.87613644,
        0.69079821, 0.8965596 ],
       [0.86004501, 0.76993247, 0.7149838 , 0.91414594, 0.98056655,
        0.9834486 , 0.78754533],
       [0.81289648, 0.60633696, 0.68145665, 0.91312938, 0.85666553,
        0.92113326, 0.89469098]])

### Другие функции агрегации

|Function Name      |   NaN-safe Version  | Description                                   |
|-------------------|---------------------|-----------------------------------------------|
| ``np.sum``        | ``np.nansum``       | Compute sum of elements                       |
| ``np.prod``       | ``np.nanprod``      | Compute product of elements                   |
| ``np.mean``       | ``np.nanmean``      | Compute mean of elements                      |
| ``np.std``        | ``np.nanstd``       | Compute standard deviation                    |
| ``np.var``        | ``np.nanvar``       | Compute variance                              |
| ``np.min``        | ``np.nanmin``       | Find minimum value                            |
| ``np.max``        | ``np.nanmax``       | Find maximum value                            |
| ``np.argmin``     | ``np.nanargmin``    | Find index of minimum value                   |
| ``np.argmax``     | ``np.nanargmax``    | Find index of maximum value                   |
| ``np.median``     | ``np.nanmedian``    | Compute median of elements                    |
| ``np.percentile`` | ``np.nanpercentile``| Compute rank-based statistics of elements     |
| ``np.any``        | N/A                 | Evaluate whether any elements are true        |
| ``np.all``        | N/A                 | Evaluate whether all elements are true        |


# Вычисления над массивами NumPy: трансляция - приведение размерностей (broadcasting)

Универсальные функции NumPy позволяют осуществлять *векторизации* операций и тем самым исключать медленные циклы Python. Бинарные операции (например, сложение, вычитание, умножение и т.д.)  принимают пару массивов и до сих пор эти массивы имели одинаковую размерность. Трансляция (broadcasting) - это просто набор правил, который примененяется к массивам разных размерностей, при вызове бинарных функций.

## Введение в трансляцию массивов

In [278]:
a_1 = np.array([0, 1, 2])
b_1 = np.array([5, 5, 5])
a_1 + b_1

array([5, 6, 7])

In [279]:
a_1 + 5

array([5, 6, 7])

In [280]:
c_2 = np.ones((3, 3))

In [281]:
c_2 + a_1

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

Как раньше мы растягивали или транслировали одно значение для соответствия форме другого, здесь мы растянули /``a_1``, и ``b_1`` для соответствия общей форме, и в результате получился двумерный массив!

![Broadcasting Visual](images/02.05-broadcasting.png)

Светлые ячейки представляют транслированые значения: дополнительная память фактически не выделяется, но концептуально может быть полезно представить, что это так.

## Правила трансляции массивов

Трансляция в NumPy подчиняется строгому набору правил, определяющих взаимодействие между двумя массивами:

- Правило 1: Если два массива различаются по количеству измерений, то форма массива с меньшим количеством измерений *дополняется* единицами на его ведущей (левой) стороне.
- Правило 2: Если форма двух массивов не совпадает в каком-либо измерении, то массив с формой, равной 1 в этом измерении, растягивается, чтобы соответствовать другой форме.
- Правило 3: Если в каком-либо измерении размеры не совпадают и ни один из них не равен 1, то выдается ошибка.

Чтобы эти правила стали понятны, рассмотрим несколько примеров подробнее.

### Пример трансляции 1

Пример, добавление двумерного массива к одномерному массиву

In [282]:
M = np.ones((2, 3))
a = np.arange(3)

Рассмотрим операцию над этими двумя массивами. Массивы имеют следующие размерности

- ``M.shape = (2, 3)``
- ``a.shape = (3,)``.

По правилу 1 мы видим, что массив ``a`` имеет меньшую размерность, поэтому мы заполняем его слева единицами:

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``.

По правилу 2, мы видим, что первое измерение не совпадает, поэтому мы растягиваем это измерение до соответствия:

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``.

Формы совпадают, и мы видим, что окончательная форма будет ``(2, 3)``:

In [283]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.]])

### Пример трансляции 2

Пример, в котором необходимо транслировать оба массива

In [284]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)

Рассмотрим операцию над этими двумя массивами. Массивы имеют следующие размерности

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``.

Правило 1 гласит, что мы должны заполнить форму ``b`` единицами:

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``.

А правило 2 говорит нам, что мы обновляем каждый из этих единичных массивов до соответствующего размера другого массива:

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``.

Поскольку результат совпадает, эти формы совместимы.

In [285]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### Пример трансляции 3

Пример, в котором два массива несовместимы

In [286]:
M = np.ones((3, 2))
a = np.arange(3)

Ситуация, отличная от первого примера: матрица ``M`` транспонирована.
Как это влияет на вычисления? Массивы имеют следующие размерности

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``.

Правило 1 говорит нам, что мы должны заполнить форму ``a`` единицами:

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``.

Правило 2, первое измерение ``a`` растягивается, чтобы соответствовать измерению ``M``:

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``.

Правило 3 - конечные формы не совпадают, поэтому эти два массива несовместимы

In [287]:
#M + a

Обратите внимание на потенциальную путаницу: можно представить, что можно сделать ``a`` и ``M`` совместимыми, скажем, дополнив форму ``a`` теми, что справа, а не слева.
Но правила трансляции работают не так!
Такая гибкость может быть полезна в некоторых случаях, но она приведет к потенциальным областям неоднозначности.
Если вам нужна правосторонняя подстановка, вы можете сделать это явно, изменив форму массива (мы будем использовать ключевое слово ``np.newaxis``, представленное в [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb)):

In [288]:
a[:, np.newaxis].shape

(3, 1)

In [289]:
M + a[:, np.newaxis]

array([[1., 1.],
       [2., 2.],
       [3., 3.]])

# Сравнение, маски и булевы операций

Рассмотрим использование булевых масок для исследования и манипулирования значениями в массивах NumPy.
Маскирование возникает, когда вы хотите извлечь, изменить, подсчитать или иным образом манипулировать значениями в массиве на основе некоторого критерия: например, вы можете захотеть подсчитать все значения, превышающие определенное значение, или удалить все отклонения, которые превышают некоторый порог.
В NumPy булево маскирование часто является наиболее эффективным способом решения подобных задач.

## Операторы сравнения, как ufuncs

NumPy реализует операторы сравнения, такие как ``<`` (меньше чем) и ``>`` (больше чем), как поэлементные *ufuncs*.
Результатом этих операторов сравнения всегда является массив с булевым типом данных.
Доступны все шесть стандартных операций сравнения.

In [290]:
x = np.array([1, 2, 3, 4, 5])

In [291]:
x < 3  # less than

array([ True,  True, False, False, False])

In [292]:
x > 3  # greater than

array([False, False, False,  True,  True])

In [293]:
x <= 3  # less than or equal

array([ True,  True,  True, False, False])

In [294]:
x >= 3  # greater than or equal

array([False, False,  True,  True,  True])

In [295]:
x != 3  # not equal

array([ True,  True, False,  True,  True])

In [296]:
x == 3  # equal

array([False, False,  True, False, False])

Можно выполнять поэлементное сравнение двух массивов, включая составные выражения

In [297]:
(2 * x) == (x ** 2)

array([False,  True, False, False, False])

| Operator	    | Equivalent ufunc    | Operator	   | Equivalent ufunc   |
|---------------|---------------------|---------------|---------------------|
|``==``         |``np.equal``         |``!=``         |``np.not_equal``     |
|``<``          |``np.less``          |``<=``         |``np.less_equal``    |
|``>``          |``np.greater``       |``>=``         |``np.greater_equal`` |

## Работа с булевыми массивами

С массивом булевых чисел можно выполнять множество полезных операций.

In [298]:
rng = np.random.RandomState(0)
x = rng.randint(10, size=(3, 4))
x

array([[5, 0, 3, 3],
       [7, 9, 3, 5],
       [2, 4, 7, 6]])

Для подсчета количества ``истинных`` записей в булевом массиве полезно использовать ``np.count_nonzero''.

In [299]:
np.count_nonzero(x < 6)

8

In [300]:
np.sum(x < 6)

8

In [301]:
np.sum(x < 6, axis=1)

array([4, 2, 2])

Если мы хотим быстро проверить, все ли значения истинны, мы можем использовать ``np.any`` или ``np.all``.

In [302]:
np.any(x > 8)

True

In [303]:
np.all(x == 6)

False

``np.all`` и ``np.any`` можно использовать вдоль заданной оси.

In [304]:
np.all(x < 8, axis=1)

array([ True, False,  True])

В Python для построения составных логических выражений используются *битовые логические операторы*, ``&``, ``|``, ``^`` и ``~`` (и, или, отрицание).
Как и в случае со стандартными арифметическими операторами, NumPy перегружает их в виде ufuncs, которые работают с массивами (обычно булевыми) поэлементно.

In [326]:
np.sum((x > 0.5) & (x < 1))

0

In [328]:
np.sum(~( (x <= 0.5) | (x >= 4) ))

3

| Operator	    | Equivalent ufunc    | Operator	    | Equivalent ufunc    |
|---------------|---------------------|---------------|---------------------|
|``&``          |``np.bitwise_and``   |``\|``          |``np.bitwise_or``    |
|``^``          |``np.bitwise_xor``   |``~``          |``np.bitwise_not``   |

## Булевы массивы в качестве масок

В предыдущем разделе мы рассмотрели агрегаты, вычисляемые непосредственно на булевых массивах.
Более мощная схема заключается в использовании булевых массивов в качестве масок для выбора определенных подмножеств данных.
Возвращаясь к нашему массиву ``x``, предположим, что нам нужен массив всех значений в массиве, которые меньше, скажем, 5.

In [305]:
x

array([[5, 0, 3, 3],
       [7, 9, 3, 5],
       [2, 4, 7, 6]])

In [306]:
x[x < 5]

array([0, 3, 3, 3, 2, 4])

## Использование ключевых слов ``and`` и ``or`` и операторов ``&`` и ``|``

Между ключевыми словами ``and`` и ``or``, с одной стороны, и операторами ``&`` и ``|``, с другой, имеется значительная разница.
Разница заключается в следующем: ``and`` и ``or`` определяют истинность или ложность *всего объекта*, в то время как ``&`` и ``|`` относятся к *битам внутри каждого объекта*.

Когда используются ``and`` или ``or``, Python объект рассматривается, как единую булеву сущность. В Python все ненулевые целые числа оцениваются как True.

In [307]:
bool(42), bool(0)

(True, False)

In [308]:
bool(42 and 0)

False

In [309]:
bool(42 or 0)

True

Когда используются ``&`` или ``|``  для целых чисел, выражения
When you use ``&`` and ``|`` on integers, the expression operates on the bits of the element, applying the *and* or the *or* to the individual bits making up the number:

In [310]:
bin(42), bin(59)

('0b101010', '0b111011')

In [311]:
bin(42 & 59), bin(42 | 59)

('0b101010', '0b111011')

When you have an array of Boolean values in NumPy, this can be thought of as a string of bits where ``1 = True`` and ``0 = False``, and the result of ``&`` and ``|`` operates similarly to above:

In [312]:
A = np.array([1, 0, 1, 0, 1, 0], dtype=bool)
B = np.array([1, 1, 1, 0, 1, 1], dtype=bool)
A | B

array([ True,  True,  True, False,  True,  True])

In [313]:
#A or B

# «Прихотливая» индексация (Fancy Indexing)

Мы рассмотрели, как получить доступ и изменить части массивов с помощью простых индексов (например, ``arr[0]``), сечения (например, ``arr[:5]``) и булевых масок (например, ``arr[arr > 0]``).
Мы рассмотрим другой стиль индексирования массивов, известный как *прихотливая индексация*.
Прихотливая индексация похожа на простую индексацию, которую мы уже видели, но вместо отдельных индексов мы передаем массивы индексов.
Что позволяет очень быстро получать доступ и изменять сложные подмножества значений массива.

## Изучение «прихотливая» индексации

In [314]:
rand = np.random.RandomState(42)

x = rand.randint(100, size=10)
print(x)

[51 92 14 71 60 20 82 86 74 74]


In [315]:
[x[3], x[7], x[2]]

[71, 86, 14]

In [316]:
ind = [3, 7, 4]
x[ind]

array([71, 86, 60])

In [317]:
ind = np.array([[3, 7],[4, 5]])
x[ind]

array([[71, 86],
       [60, 20]])

In [334]:
X = np.arange(12).reshape((3, 4))
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [335]:
row = np.array([0, 1, 2])
col = np.array([2, 1, 3])
X[row, col]

array([ 2,  5, 11])

In [336]:
X[row[:, np.newaxis], col]

array([[ 2,  1,  3],
       [ 6,  5,  7],
       [10,  9, 11]])

In [337]:
row[:, np.newaxis] * col

array([[0, 0, 0],
       [2, 1, 3],
       [4, 2, 6]])

## Комбинированная индексация

In [338]:
X[2, [2, 0, 1]]

array([10,  8,  9])

In [339]:
X[1:, [2, 0, 1]]

array([[ 6,  4,  5],
       [10,  8,  9]])

In [340]:
mask = np.array([1, 0, 1, 0], dtype=bool)
X[row[:, np.newaxis], mask]

array([[ 0,  2],
       [ 4,  6],
       [ 8, 10]])

# Сортировка массивов

## Быстрая сортировка в NumPy: ``np.sort`` и ``np.argsort``

B Python есть встроенные функции ``sort`` и ``sorted`` для работы со списками, однако функция NumPy ``np.sort`` оказывается гораздо более эффективной и полезной для наших задач.

In [320]:
x = np.array([2, 1, 4, 3, 5])
np.sort(x)

array([1, 2, 3, 4, 5])

In [321]:
x.sort()
x

array([1, 2, 3, 4, 5])

In [322]:
x = np.array([2, 1, 4, 3, 5])
i = np.argsort(x)
print(i)

[1 0 3 2 4]


### Сортировка вдоль строки и колонки

Полезной особенностью алгоритмов сортировки NumPy является возможность сортировки вдоль осей многомерного массива с помощью аргумента ``axis``.

In [323]:
rand = np.random.RandomState(42)
X = rand.randint(0, 10, (4, 6))
print(X)

[[6 3 7 4 6 9]
 [2 6 7 4 3 7]
 [7 2 5 4 1 7]
 [5 1 4 0 9 5]]


In [324]:
# sort each column of X
np.sort(X, axis=0)

array([[2, 1, 4, 0, 1, 5],
       [5, 2, 5, 4, 3, 7],
       [6, 3, 7, 4, 6, 7],
       [7, 6, 7, 4, 9, 9]])

In [325]:
# sort each row of X
np.sort(X, axis=1)

array([[3, 4, 6, 6, 7, 9],
       [2, 3, 4, 6, 7, 7],
       [1, 2, 4, 5, 7, 7],
       [0, 1, 4, 5, 5, 9]])